## Libraries

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import cv2 as cv
import numpy as np
import random

from xml.etree import ElementTree
from xml.dom import minidom
import collections

import os

import matplotlib.pyplot as plt
import matplotlib as matplot
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split

In [ ]:
def f1_score(dict):
  f1_avg = 0
  for i in dict.keys():
    f1 = (2*dict[i][0]*dict[i][1])/(dict[i][0]+dict[i][1])
    print(i,round(f1,3))
    f1_avg += f1
  print(round(f1_avg/4,3))

In [ ]:
def label_inspector(file_list):

  cls_names = []
  total_images = 0

  for file in file_list:
    total_images = total_images + 1

    file = file.rstrip('\n').split('/')[-1]
    
    file = file.split('.')[0] + '.xml'

    infile_xml = open(base_path + '/annotations/xmls/' + file)
    tree = ElementTree.parse(infile_xml)
    root = tree.getroot()
    main_labels = []

    for obj in root.iter('object'):
      cls_name = obj.find('name').text
      #if cls_name in mainTypes:
      cls_names.append(cls_name)

  print("total")
  print("# of images：" + str(total_images))
  print("# of labels：" + str(len(cls_names)))


  count_dict = collections.Counter(cls_names)
  cls_count = []
  for damageType in mainTypes:
      print(str(damageType) + ' : ' + str(count_dict[damageType]))
      cls_count.append(count_dict[damageType])
      
  sns.set_palette("winter", 8)
  sns.barplot(mainTypes, cls_count)


In [ ]:
 %cd gdrive/My Drive/ultralytics
#! git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt  # install

/content/gdrive/My Drive/ultralytics
/content/gdrive/My Drive/ultralytics/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 8.2 MB/s 


In [ ]:
base_path = os.getcwd() + '/datasets/roadColombia/Colombia'
damageTypes=["D00", "D01", "D10", "D11", "D20", "D40", "D43", "D44"]
mainTypes=["D00", "D10", "D20", "D40"]

## Transfer learning

### Experiment 1

In [ ]:
!python3 train.py --data data/road_col.yml --weights weights/IMSC/last_95.pt --img 640 --batch-size 32 --epochs 80 --freeze 24

train: weights=weights/IMSC/last_95.pt, cfg=, data=data/road_col.yml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=80, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[24], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 373 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj

In [ ]:
dict = {'D00':[0.454,0.308],
        'D10':[0.506,0.385],
        'D20':[0.462,0.238],
        'D40':[0.379,0.169]}

f1_score(dict)

D00 0.367
D10 0.437
D20 0.314
D40 0.234
0.338


In [ ]:
!python val.py --weights runs/train/exp/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml --augment --half --task test # --name exp

val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml, weights=['runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=test, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 484 layers, 88410801 parameters, 0 gradients, 219.6 GFLOPs
test: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/test.cache' images and labels... 117 found, 0 missing, 0 empty, 0 corrupt: 100% 117/117 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 4/4 [01:03<00:00, 15.78s/it]
                 all        117        269      0.417      0.325      0.259       0.11
                 D00        11

In [ ]:
dict = {'D00':[0.59,0.488],
        'D10':[0.365,0.316],
        'D20':[0.456,0.31],
        'D40':[0.255,0.187]}

f1_score(dict)

D00 0.534
D10 0.339
D20 0.369
D40 0.216
0.364


In [ ]:
### train metrics
!python val.py --weights runs/train/exp/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml --augment --half --task train --name exp1train 

val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml, weights=['runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=train, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp1train, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CPU

Fusing layers... 
Model summary: 484 layers, 88410801 parameters, 0 gradients, 219.6 GFLOPs
train: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/train' images and labels...526 found, 0 missing, 1 empty, 0 corrupt: 100% 526/526 [05:41<00:00,  1.54it/s]
train: New cache created: /content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/train.cache
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95:   0% 0/17 [00:00<?, ?it/s]tcmalloc: large alloc 1156055040 bytes == 

In [ ]:
dict = {'D00':[0.373 ,0.385],
        'D10':[0.333 ,0.385],
        'D20':[0.466,0.288],
        'D40':[ 0.3,0.144]}

f1_score(dict)

D00 0.379
D10 0.357
D20 0.356
D40 0.195
0.322


### Experiment 2

In [ ]:
!python3 train.py --data data/road_col.yml --weights weights/IMSC/last_95.pt --img 640 --batch-size 32 --epochs 80 --freeze 18

train: weights=weights/IMSC/last_95.pt, cfg=, data=data/road_col.yml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=80, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[18], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 373 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj

In [ ]:
dict = {'D00':[0.359,0.385],
        'D10':[0.663,0.462],
        'D20':[0.506,0.552],
        'D40':[0.67,0.482]}

f1_score(dict)

D00 0.372
D10 0.545
D20 0.528
D40 0.561
0.501


In [ ]:
!python val.py --weights runs/train/exp2/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml --augment --half --task test # --name exp

val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml, weights=['runs/train/exp2/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=test, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 484 layers, 88410801 parameters, 0 gradients, 219.6 GFLOPs
test: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/test.cache' images and labels... 117 found, 0 missing, 0 empty, 0 corrupt: 100% 117/117 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 4/4 [00:04<00:00,  1.10s/it]
                 all        117        269      0.516      0.398       0.43      0.209
                 D00        1

In [ ]:
dict = {'D00':[0.526,0.465],
        'D10':[0.481,0.245],
        'D20':[0.461,0.393],
        'D40':[0.596,0.488]}

f1_score(dict)

D00 0.494
D10 0.325
D20 0.424
D40 0.537
0.445


In [ ]:
### train metrics
!python val.py --weights runs/train/exp2/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml --augment --half --task train --name exp2train

val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml, weights=['runs/train/exp2/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=train, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp2train, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 484 layers, 88410801 parameters, 0 gradients, 219.6 GFLOPs
train: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/train.cache' images and labels... 526 found, 0 missing, 0 empty, 0 corrupt: 100% 526/526 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 17/17 [00:11<00:00,  1.47it/s]
                 all        526       1204      0.719      0.722      0.765      0.431
                 D

In [ ]:
dict = {'D00':[0.636,0.763],
        'D10':[0.717,0.56],
        'D20':[0.739,0.819],
        'D40':[0.781,0.744]}

f1_score(dict)

D00 0.694
D10 0.629
D20 0.777
D40 0.762
0.715


### Experiment 3

In [ ]:
!python3 train.py --data data/road_col.yml --weights weights/IMSC/last_95.pt --img 640 --batch-size 32 --epochs 80 --freeze 12

train: weights=weights/IMSC/last_95.pt, cfg=, data=data/road_col.yml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=80, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[12], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 373 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj

In [ ]:
dict = {'D00':[0.404,0.487],
        'D10':[0.76,0.462],
        'D20':[0.569,0.533],
        'D40':[0.696,0.596]}

f1_score(dict)

D00 0.442
D10 0.575
D20 0.55
D40 0.642
0.552


In [ ]:
!python val.py --weights runs/train/exp3/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml --augment --half --task test # --name exp

val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml, weights=['runs/train/exp3/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=test, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 484 layers, 88410801 parameters, 0 gradients, 219.6 GFLOPs
test: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/test.cache' images and labels... 117 found, 0 missing, 0 empty, 0 corrupt: 100% 117/117 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 4/4 [00:04<00:00,  1.06s/it]
                 all        117        269      0.618      0.435      0.506      0.244
                 D00        1

In [ ]:
dict = {'D00':[0.57,0.465],
        'D10':[0.604,0.211],
        'D20':[0.612,0.44],
        'D40':[0.686,0.623]}

f1_score(dict)

D00 0.512
D10 0.313
D20 0.512
D40 0.653
0.497


In [ ]:
### train metrics
!python val.py --weights runs/train/exp3/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml --augment --half --task train --name exp3train

val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml, weights=['runs/train/exp3/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=train, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp3train, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 484 layers, 88410801 parameters, 0 gradients, 219.6 GFLOPs
train: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/train.cache' images and labels... 526 found, 0 missing, 0 empty, 0 corrupt: 100% 526/526 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 17/17 [00:11<00:00,  1.52it/s]
                 all        526       1204      0.754      0.721      0.795       0.44
                 D

In [ ]:
dict = {'D00':[0.683,0.737],
        'D10':[0.802,0.495],
        'D20':[0.8,0.822],
        'D40':[0.732,0.83]}

f1_score(dict)

D00 0.709
D10 0.612
D20 0.811
D40 0.778
0.727


### Experiment 4

In [ ]:
!python3 train.py --data data/road_col.yml --weights weights/IMSC/last_95.pt --img 640 --batch-size 32 --epochs 80 --freeze 6

train: weights=weights/IMSC/last_95.pt, cfg=, data=data/road_col.yml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=80, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[6], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 373 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_

In [ ]:
dict = {'D00':[0.428,0.333],
        'D10':[0.557,0.484],
        'D20':[0.705,0.455],
        'D40':[0.736,0.56]}

f1_score(dict)

D00 0.375
D10 0.518
D20 0.553
D40 0.636
0.52


In [ ]:
!python val.py --weights runs/train/exp4/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml --augment --half --task test # --name exp

val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml, weights=['runs/train/exp4/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=test, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 484 layers, 88410801 parameters, 0 gradients, 219.6 GFLOPs
test: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/test.cache' images and labels... 117 found, 0 missing, 0 empty, 0 corrupt: 100% 117/117 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 4/4 [00:03<00:00,  1.01it/s]
                 all        117        269      0.537      0.437      0.476      0.232
                 D00        1

In [ ]:
dict = {'D00':[0.564,0.395],
        'D10':[0.407,0.316],
        'D20':[0.557,0.419],
        'D40':[0.619,0.618]}

f1_score(dict)

D00 0.465
D10 0.356
D20 0.478
D40 0.618
0.479


In [ ]:
### train metrics
!python val.py --weights runs/train/exp4/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml --augment --half --task train --name exp4train

val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml, weights=['runs/train/exp4/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=train, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp4train, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 484 layers, 88410801 parameters, 0 gradients, 219.6 GFLOPs
train: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/train.cache' images and labels... 526 found, 0 missing, 0 empty, 0 corrupt: 100% 526/526 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 17/17 [00:10<00:00,  1.55it/s]
                 all        526       1204      0.922      0.942       0.97      0.664
                 D

In [ ]:
dict = {'D00':[0.939,0.986],
        'D10':[0.86,0.901],
        'D20':[0.973,0.973],
        'D40':[0.918,0.909]}

f1_score(dict)

D00 0.962
D10 0.88
D20 0.973
D40 0.913
0.932


### Experiment 5

In [ ]:
!python3 train.py --data data/road_col.yml --weights yolov5s.pt --img 640 --batch-size 32 --epochs 80 --freeze 12

train: weights=yolov5s.pt, cfg=, data=data/road_col.yml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=80, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[12], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 373 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_

In [ ]:
dict = {'D00':[0.333,0.256],
        'D10':[0.143,0.0385],
        'D20':[0.502,0.39],
        'D40':[0.572,0.464]}

f1_score(dict)

D00 0.289
D10 0.061
D20 0.439
D40 0.512
0.325


In [ ]:
!python val.py --weights runs/train/exp5/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml --augment --half --task test # --name exp

val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml, weights=['runs/train/exp5/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=test, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 213 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/test.cache' images and labels... 117 found, 0 missing, 0 empty, 0 corrupt: 100% 117/117 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 4/4 [00:03<00:00,  1.02it/s]
                 all        117        269      0.412      0.311      0.293      0.129
                 D00        117

In [ ]:
dict = {'D00':[0.311,0.302],
        'D10':[0.389,0.0526],
        'D20':[0.375,0.345],
        'D40':[0.572,0.543]}

f1_score(dict)

D00 0.306
D10 0.093
D20 0.359
D40 0.557
0.329


In [ ]:
### train metrics
!python val.py --weights runs/train/exp5/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml --augment --half --task train --name exp5train

val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml, weights=['runs/train/exp5/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=train, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp5train, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 213 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
train: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/train.cache' images and labels... 526 found, 0 missing, 1 empty, 0 corrupt: 100% 526/526 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 17/17 [00:07<00:00,  2.27it/s]
                 all        526       1203      0.738      0.716      0.805      0.448
                 D00

In [ ]:
dict = {'D00':[0.591, 0.842],
        'D10':[0.839 ,0.287],
        'D20':[0.72 ,0.856 ],
        'D40':[0.8,0.879]}

f1_score(dict)

D00 0.695
D10 0.428
D20 0.782
D40 0.838
0.685


### Experiment 6

In [ ]:
!python3 train.py --data data/road_col.yml --weights weights/IMSC/last_95.pt --img 640 --batch-size 32 --epochs 80

train: weights=weights/IMSC/last_95.pt, cfg=, data=data/road_col.yml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=80, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 373 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_

In [ ]:
dict = {'D00':[0.391,0.487],
        'D10':[0.589,0.462],
        'D20':[0.503,0.505],
        'D40':[0.679,0.61]}

f1_score(dict)

D00 0.434
D10 0.518
D20 0.504
D40 0.643
0.525


In [ ]:
!python val.py --weights runs/train/exp6/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml --augment --half --task test # --name exp

val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml, weights=['runs/train/exp6/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=test, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 484 layers, 88410801 parameters, 0 gradients, 219.6 GFLOPs
test: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/test.cache' images and labels... 117 found, 0 missing, 0 empty, 0 corrupt: 100% 117/117 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 4/4 [00:03<00:00,  1.03it/s]
                 all        117        269      0.509      0.512      0.488      0.226
                 D00        1

In [ ]:
dict = {'D00':[0.468,0.442],
        'D10':[0.487,0.45],
        'D20':[0.504,0.548],
        'D40':[0.577,0.61]}

f1_score(dict)

D00 0.455
D10 0.468
D20 0.525
D40 0.593
0.51


In [ ]:
### train metrics
!python val.py --weights runs/train/exp6/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml --augment --half --task train --name exp6train


val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml, weights=['runs/train/exp6/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=train, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp6train, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 484 layers, 88410801 parameters, 0 gradients, 219.6 GFLOPs
train: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/train.cache' images and labels... 526 found, 0 missing, 1 empty, 0 corrupt: 100% 526/526 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 17/17 [00:10<00:00,  1.56it/s]
                 all        526       1203      0.931      0.951      0.978      0.694
                 D

In [ ]:
dict = {'D00':[0.918,0.987],
        'D10':[0.92 ,0.885],
        'D20':[0.923,0.995],
        'D40':[0.964,0.936 ]}

f1_score(dict)

D00 0.951
D10 0.902
D20 0.958
D40 0.95
0.94


### Experiment 7

In [ ]:
!python3 train.py --data data/road_col.yml --weights weights/IMSC/last_95.pt --img 640 --batch-size 32 --epochs 150

train: weights=weights/IMSC/last_95.pt, cfg=, data=data/road_col.yml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=150, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 373 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj

In [ ]:
dict = {'D00':[0.472,0.41],
        'D10':[0.64,0.478],
        'D20':[0.531,0.486],
        'D40':[0.672,0.61]}

f1_score(dict)

D00 0.439
D10 0.547
D20 0.508
D40 0.64
0.533


In [ ]:
!python val.py --weights runs/train/exp7/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml --augment --half --task test # --name exp

val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml, weights=['runs/train/exp7/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=test, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 484 layers, 88410801 parameters, 0 gradients, 219.6 GFLOPs
test: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/test.cache' images and labels... 117 found, 0 missing, 0 empty, 0 corrupt: 100% 117/117 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 4/4 [00:03<00:00,  1.05it/s]
                 all        117        269      0.448       0.52      0.477      0.226
                 D00        1

In [ ]:
dict = {'D00':[0.443,0.463],
        'D10':[0.365,0.474],
        'D20':[0.444,0.524],
        'D40':[0.539,0.618]}

f1_score(dict)

D00 0.453
D10 0.412
D20 0.481
D40 0.576
0.48


In [ ]:
### train metrics
!python val.py --weights runs/train/exp7/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml --augment --half --task train --name exp7train


val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml, weights=['runs/train/exp7/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=train, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp7train, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 484 layers, 88410801 parameters, 0 gradients, 219.6 GFLOPs
train: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/train.cache' images and labels... 526 found, 0 missing, 1 empty, 0 corrupt: 100% 526/526 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 17/17 [00:10<00:00,  1.56it/s]
                 all        526       1203      0.965      0.958      0.988      0.733
                 D

In [ ]:
dict = {'D00':[ 0.965,0.994],
        'D10':[0.964 ,0.893],
        'D20':[0.975,0.989],
        'D40':[0.957 ,0.955 ]}

f1_score(dict)

D00 0.979
D10 0.927
D20 0.982
D40 0.956
0.961


### Experiment 8

In [ ]:
!python3 train.py --data data/road_col.yml --weights weights/IMSC/last_95.pt --img 640 --batch-size 32 --epochs 40

train: weights=weights/IMSC/last_95.pt, cfg=, data=data/road_col.yml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=40, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 373 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_

In [ ]:
dict = {'D00':[0.455,0.538],
        'D10':[0.806,0.482],
        'D20':[0.674,0.448],
        'D40':[0.718,0.532]}

f1_score(dict)

D00 0.493
D10 0.603
D20 0.538
D40 0.611
0.561


In [ ]:
!python val.py --weights runs/train/exp8/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml --augment --half --task test # --name exp

val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml, weights=['runs/train/exp8/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=test, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 484 layers, 88410801 parameters, 0 gradients, 219.6 GFLOPs
test: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/test.cache' images and labels... 117 found, 0 missing, 0 empty, 0 corrupt: 100% 117/117 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 4/4 [00:04<00:00,  1.01s/it]
                 all        117        269      0.525      0.537      0.542      0.262
                 D00        1

In [ ]:
dict = {'D00':[0.464,0.488],
        'D10':[0.465,0.421],
        'D20':[0.589,0.607],
        'D40':[0.581,0.631]}

f1_score(dict)

D00 0.476
D10 0.442
D20 0.598
D40 0.605
0.53


In [ ]:
### train metrics
!python val.py --weights runs/train/exp8/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml --augment --half --task train --name exp8train

val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml, weights=['runs/train/exp8/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=train, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp8train, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 484 layers, 88410801 parameters, 0 gradients, 219.6 GFLOPs
train: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/train.cache' images and labels... 526 found, 0 missing, 0 empty, 0 corrupt: 100% 526/526 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 17/17 [00:10<00:00,  1.57it/s]
                 all        526       1204      0.843      0.819      0.888      0.538
                 D

In [ ]:
dict = {'D00':[0.811,0.897],
        'D10':[0.826,0.678],
        'D20':[0.871,0.894],
        'D40':[0.864,0.806]}

f1_score(dict)

D00 0.852
D10 0.745
D20 0.882
D40 0.834
0.828


### Experiment 9

In [ ]:
!python3 train.py --data data/road_col.yml --weights weights/IMSC/last_95.pt --img 640 --batch-size 16 --epochs 40

train: weights=weights/IMSC/last_95.pt, cfg=, data=data/road_col.yml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=40, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 373 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_

In [ ]:
dict = {'D00':[0.425,0.513],
        'D10':[0.773,0.5],
        'D20':[0.568,0.457],
        'D40':[0.701,0.574]}

f1_score(dict)

D00 0.465
D10 0.607
D20 0.506
D40 0.631
0.552


In [ ]:
!python val.py --weights runs/train/exp9/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml --augment --half --task test # --name exp

val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml, weights=['runs/train/exp9/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=test, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 484 layers, 88410801 parameters, 0 gradients, 219.6 GFLOPs
test: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/test.cache' images and labels... 117 found, 0 missing, 0 empty, 0 corrupt: 100% 117/117 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 4/4 [00:04<00:00,  1.03s/it]
                 all        117        269      0.515      0.475      0.515      0.255
                 D00        1

In [ ]:
dict = {'D00':[0.551,0.429],
        'D10':[0.388,0.316],
        'D20':[0.532,0.512],
        'D40':[0.59,0.644]}

f1_score(dict)

D00 0.482
D10 0.348
D20 0.522
D40 0.616
0.492


In [ ]:
### train metrics
!python val.py --weights runs/train/exp9/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml --augment --half --task train --name exp9train


val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml, weights=['runs/train/exp9/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=train, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp9train, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 484 layers, 88410801 parameters, 0 gradients, 219.6 GFLOPs
train: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/train.cache' images and labels... 526 found, 0 missing, 1 empty, 0 corrupt: 100% 526/526 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 17/17 [00:10<00:00,  1.55it/s]
                 all        526       1203      0.812      0.849      0.893       0.55
                 D

In [ ]:
dict = {'D00':[0.769,0.917],
        'D10':[0.765 ,0.714],
        'D20':[0.869,0.912],
        'D40':[0.843 ,0.854 ]}

f1_score(dict)

D00 0.837
D10 0.739
D20 0.89
D40 0.848
0.828


### Experiment 10

In [ ]:
!python3 train.py --data data/road_col.yml --weights weights/IMSC/last_95.pt --img 640 --batch-size 48 --epochs 40

train: weights=weights/IMSC/last_95.pt, cfg=, data=data/road_col.yml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=40, batch_size=48, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 373 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_

In [ ]:
dict = {'D00':[0.476,0.487],
        'D10':[0.836,0.462],
        'D20':[0.664,0.419],
        'D40':[0.725,0.489]}

f1_score(dict)

D00 0.481
D10 0.595
D20 0.514
D40 0.584
0.544


In [ ]:
!python val.py --weights runs/train/exp10/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml --augment --half --task test # --name exp

val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml, weights=['runs/train/exp10/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=test, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 484 layers, 88410801 parameters, 0 gradients, 219.6 GFLOPs
test: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/test.cache' images and labels... 117 found, 0 missing, 0 empty, 0 corrupt: 100% 117/117 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 4/4 [00:04<00:00,  1.01s/it]
                 all        117        269      0.559      0.456      0.514      0.261
                 D00        

In [ ]:
dict = {'D00':[0.576,0.465],
        'D10':[0.417,0.263],
        'D20':[0.573,0.536],
        'D40':[0.672,0.561]}

f1_score(dict)

D00 0.515
D10 0.323
D20 0.554
D40 0.612
0.501


In [ ]:
### train metrics
!python val.py --weights runs/train/exp10/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml --augment --half --task train --name exp10train


val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml, weights=['runs/train/exp10/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=train, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp10train, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 484 layers, 88410801 parameters, 0 gradients, 219.6 GFLOPs
train: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/train.cache' images and labels... 526 found, 0 missing, 1 empty, 0 corrupt: 100% 526/526 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 17/17 [00:10<00:00,  1.56it/s]
                 all        526       1203      0.801       0.82      0.869      0.508
                

In [ ]:
dict = {'D00':[0.76,0.891 ],
        'D10':[ 0.781,0.703],
        'D20':[0.824,0.882],
        'D40':[0.839,0.806]}

f1_score(dict)

D00 0.82
D10 0.74
D20 0.852
D40 0.822
0.809


### Experiment 11

In [ ]:
  !python3 train.py --data data/road_col.yml --weights yolov5s.pt --img 640 --batch-size 32 --epochs 40

train: weights=yolov5s.pt, cfg=, data=data/road_col.yml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=40, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 373 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t

In [ ]:
dict = {'D00':[0.433,0.392],
        'D10':[0.631,0.308],
        'D20':[0.621,0.448],
        'D40':[0.745,0.61]}

f1_score(dict)

D00 0.411
D10 0.414
D20 0.521
D40 0.671
0.504


In [ ]:
!python val.py --weights runs/train/exp11/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml --augment --half --task test # --name exp

val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml, weights=['runs/train/exp11/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=test, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 213 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/test.cache' images and labels... 117 found, 0 missing, 0 empty, 0 corrupt: 100% 117/117 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 4/4 [00:03<00:00,  1.11it/s]
                 all        117        269      0.392      0.417      0.387      0.184
                 D00        11

In [ ]:
dict = {'D00':[0.32,0.372],
        'D10':[0.301,0.211],
        'D20':[0.389,0.405],
        'D40':[0.559,0.681]}

f1_score(dict)

D00 0.344
D10 0.248
D20 0.397
D40 0.614
0.401


In [ ]:
### train metrics
!python val.py --weights runs/train/exp11/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml --augment --half --task train --name exp11train

val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml, weights=['runs/train/exp11/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=train, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp11train, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 213 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
train: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/train.cache' images and labels... 526 found, 0 missing, 0 empty, 0 corrupt: 100% 526/526 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 17/17 [00:07<00:00,  2.33it/s]
                 all        526       1204      0.828      0.844      0.905       0.53
                 D

In [ ]:
dict = {'D00':[0.755,0.904],
        'D10':[0.831,0.648],
        'D20':[0.868,0.91],
        'D40':[0.856,0.912]}

f1_score(dict)

D00 0.823
D10 0.728
D20 0.889
D40 0.883
0.831


### Experiment 12

In [ ]:
!python3 train.py --data data/road_col.yml --weights yolov5x.pt --img 640 --batch-size 32 --epochs 40

train: weights=yolov5x.pt, cfg=, data=data/road_col.yml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=40, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 373 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t

In [ ]:
dict = {'D00':[0.513,0.436],
        'D10':[0.61,0.423],
        'D20':[0.673,0.457],
        'D40':[0.742,0.631]}

f1_score(dict)

D00 0.471
D10 0.5
D20 0.544
D40 0.682
0.549


In [ ]:
!python val.py --weights runs/train/exp12/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml --augment --half --task test # --name exp

val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml, weights=['runs/train/exp12/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=test, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 444 layers, 86193601 parameters, 0 gradients, 203.8 GFLOPs
test: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/test.cache' images and labels... 117 found, 0 missing, 0 empty, 0 corrupt: 100% 117/117 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 4/4 [00:03<00:00,  1.02it/s]
                 all        117        269      0.571      0.455      0.469      0.229
                 D00        

In [ ]:
dict = {'D00':[0.586,0.428],
        'D10':[0.538,0.457],
        'D20':[0.517,0.5],
        'D40':[0.643,0.577]}

f1_score(dict)

D00 0.495
D10 0.494
D20 0.508
D40 0.608
0.526


In [ ]:
### train metrics
!python val.py --weights runs/train/exp12/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml --augment --half --task train --name exp12train

val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml, weights=['runs/train/exp12/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=train, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp12train, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 444 layers, 86193601 parameters, 0 gradients, 203.8 GFLOPs
train: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/train.cache' images and labels... 526 found, 0 missing, 0 empty, 0 corrupt: 100% 526/526 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 17/17 [00:10<00:00,  1.64it/s]
                 all        526       1204      0.956      0.955      0.986      0.712
                

In [ ]:
dict = {'D00':[0.935,0.987],
        'D10':[0.958,0.868],
        'D20':[0.964,0.992],
        'D40':[0.965,0.974]}

f1_score(dict)

D00 0.96
D10 0.911
D20 0.978
D40 0.969
0.955


### Experiment 13

In [ ]:
!python3 train.py --data data/road_col.yml --weights weights/IMSC/last_95.pt --img 640 --batch-size 32 --epochs 40 --freeze 12

train: weights=weights/IMSC/last_95.pt, cfg=, data=data/road_col.yml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=40, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[12], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 373 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj

In [ ]:
dict = {'D00':[0.374,0.462],
        'D10':[0.634,0.462],
        'D20':[0.483,0.533],
        'D40':[0.639,0.631]}

f1_score(dict)

D00 0.413
D10 0.535
D20 0.507
D40 0.635
0.522


In [ ]:
!python val.py --weights runs/train/exp13/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml --augment --half --task test # --name exp

val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml, weights=['runs/train/exp13/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=test, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 484 layers, 88410801 parameters, 0 gradients, 219.6 GFLOPs
test: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/test.cache' images and labels... 117 found, 0 missing, 0 empty, 0 corrupt: 100% 117/117 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 4/4 [00:04<00:00,  1.05s/it]
                 all        117        269      0.462      0.515      0.499      0.249
                 D00        

In [ ]:
dict = {'D00':[0.49,0.512],
        'D10':[0.402,0.316],
        'D20':[0.405,0.526],
        'D40':[0.551,0.707]}

f1_score(dict)

D00 0.501
D10 0.354
D20 0.458
D40 0.619
0.483


In [ ]:
### train metrics
!python val.py --weights runs/train/exp13/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml --augment --half --task train --name exp13train

val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col.yml, weights=['runs/train/exp13/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=train, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp13train, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 484 layers, 88410801 parameters, 0 gradients, 219.6 GFLOPs
train: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/train.cache' images and labels... 526 found, 0 missing, 0 empty, 0 corrupt: 100% 526/526 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 17/17 [00:10<00:00,  1.55it/s]
                 all        526       1204      0.764      0.767      0.825      0.483
                

In [ ]:
dict = {'D00':[0.751,0.733],
        'D10':[0.761,0.596],
        'D20':[0.77,0.878],
        'D40':[0.775,0.861]}

f1_score(dict)

D00 0.742
D10 0.668
D20 0.82
D40 0.816
0.762


### Experiment 14


In [ ]:
!python3 train.py --data data/road_col_extended.yml --weights weights/IMSC/last_95.pt --img 640 --batch-size 32 --epochs 80 --freeze 12

train: weights=weights/IMSC/last_95.pt, cfg=, data=data/road_col_extended.yml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=80, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[12], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
Command 'git fetch && git config --get remote.origin.url' timed out after 5 seconds
fatal: cannot change to '/content/gdrive/My': No such file or directory
YOLOv5 🚀 2022-6-28 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0

In [ ]:
dict = {'D00':[0.397,0.41 ],
        'D10':[0.628,0.455],
        'D20':[0.553,0.529],
        'D40':[0.688 ,0.547]}

f1_score(dict)

D00 0.403
D10 0.528
D20 0.541
D40 0.609
0.52


In [ ]:
!python val.py --weights runs/train/exp14/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col_extended.yml --augment --half --task test # --name exp

val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col_extended.yml, weights=['runs/train/exp14/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=test, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 484 layers, 88410801 parameters, 0 gradients, 219.6 GFLOPs
test: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/test.cache' images and labels... 117 found, 0 missing, 0 empty, 0 corrupt: 100% 117/117 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 4/4 [00:49<00:00, 12.41s/it]
                 all        117        269      0.469      0.488      0.435      0.213
                 D0

In [ ]:
dict = {'D00':[0.329,0.468],
        'D10':[0.499,0.421],
        'D20':[0.455,0.476],
        'D40':[0.591,0.585]}

f1_score(dict)

D00 0.386
D10 0.457
D20 0.465
D40 0.588
0.474


In [ ]:
### train metrics
!python val.py --weights runs/train/exp14/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col_extended.yml --augment --half --task train --name exp14train

val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col_extended.yml, weights=['runs/train/exp14/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=train, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp14train, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 484 layers, 88410801 parameters, 0 gradients, 219.6 GFLOPs
train: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/train_extended.cache' images and labels... 1526 found, 0 missing, 417 empty, 0 corrupt: 100% 1526/1526 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 48/48 [00:25<00:00,  1.89it/s]
                 all       1526       2397      0.854      0.854      0.927     

In [ ]:
dict = {'D00':[0.778,0.961],
        'D10':[0.851,0.646],
        'D20':[0.918,0.93],
        'D40':[0.871,0.88]}

f1_score(dict)

D00 0.86
D10 0.734
D20 0.924
D40 0.875
0.848


### Experiment 15

In [ ]:
!python3 train.py --data data/road_col_extended.yml --weights weights/IMSC/last_95.pt --img 640 --batch-size 32 --epochs 40 --freeze 12

train: weights=weights/IMSC/last_95.pt, cfg=, data=data/road_col_extended.yml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=40, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[12], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 376 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj

In [ ]:
dict = {'D00':[0.641,0.487],
        'D10':[0.512,0.423],
        'D20':[0.648,0.509],
        'D40':[0.682,0.624]}

f1_score(dict)

D00 0.553
D10 0.463
D20 0.57
D40 0.652
0.56


In [ ]:
!python val.py --weights runs/train/exp15/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col_extended.yml --augment --half --task test # --name exp

val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col_extended.yml, weights=['runs/train/exp15/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=test, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 484 layers, 88410801 parameters, 0 gradients, 219.6 GFLOPs
test: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/test.cache' images and labels... 117 found, 0 missing, 0 empty, 0 corrupt: 100% 117/117 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 4/4 [00:04<00:00,  1.04s/it]
                 all        117        269      0.641      0.436      0.501      0.226
                 D0

In [ ]:
dict = {'D00':[0.637, 0.395 ],
        'D10':[0.621, 0.316],
        'D20':[0.621, 0.417],
        'D40':[0.683, 0.614]}

f1_score(dict)

D00 0.488
D10 0.419
D20 0.499
D40 0.647
0.513


In [ ]:
### train metrics
!python val.py --weights runs/train/exp15/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col_extended.yml --augment --half --task train --name exp15train

val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col_extended.yml, weights=['runs/train/exp15/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=train, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp15train, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 484 layers, 88410801 parameters, 0 gradients, 219.6 GFLOPs
train: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/train_extended.cache' images and labels... 1526 found, 0 missing, 417 empty, 0 corrupt: 100% 1526/1526 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 48/48 [00:25<00:00,  1.88it/s]
                 all       1526       2397      0.848      0.853      0.917     

In [ ]:
dict = {'D00':[0.857, 0.92],
        'D10':[0.823, 0.681],
        'D20':[0.91, 0.909],
        'D40':[0.801, 0.902]}

f1_score(dict)

D00 0.887
D10 0.745
D20 0.909
D40 0.849
0.848


### Experiment 16

In [ ]:
!python3 train.py --data data/road_col_extended.yml --weights weights/IMSC/last_95.pt --img 640 --batch-size 32 --epochs 40 --freeze 18

train: weights=weights/IMSC/last_95.pt, cfg=, data=data/road_col_extended.yml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=40, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[18], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 376 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj

In [ ]:
dict = {'D00':[0.453,0.41],
        'D10':[0.608,0.346],
        'D20':[0.467,0.552],
        'D40':[0.527,0.536]}

f1_score(dict)

D00 0.43
D10 0.441
D20 0.506
D40 0.531
0.477


In [ ]:
!python val.py --weights runs/train/exp16/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col_extended.yml --augment --half --task test # --name exp

val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col_extended.yml, weights=['runs/train/exp16/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=test, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 484 layers, 88410801 parameters, 0 gradients, 219.6 GFLOPs
test: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/test.cache' images and labels... 117 found, 0 missing, 0 empty, 0 corrupt: 100% 117/117 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 4/4 [00:04<00:00,  1.09s/it]
                 all        117        269      0.543       0.39      0.424      0.186
                 D0

In [ ]:
dict = {'D00':[0.639, 0.329 ],
        'D10':[0.554, 0.263],
        'D20':[0.525, 0.435 ],
        'D40':[0.453, 0.532]}

f1_score(dict)

D00 0.434
D10 0.357
D20 0.476
D40 0.489
0.439


In [ ]:
### train metrics
!python val.py --weights runs/train/exp16/weights/best.pt --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col_extended.yml --augment --half --task train --name exp16train

val: data=/content/gdrive/MyDrive/ultralytics/yolov5/data/road_col_extended.yml, weights=['runs/train/exp16/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=train, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp16train, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-269-gf76a78e7 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536MiB)

Fusing layers... 
Model summary: 484 layers, 88410801 parameters, 0 gradients, 219.6 GFLOPs
train: Scanning '/content/gdrive/MyDrive/ultralytics/yolov5/datasets/roadColombia/train_extended.cache' images and labels... 1526 found, 0 missing, 417 empty, 0 corrupt: 100% 1526/1526 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 48/48 [00:26<00:00,  1.84it/s]
                 all       1526       2397      0.688      0.645       0.71     

In [ ]:
dict = {'D00':[0.686, 0.659],
        'D10':[0.725, 0.489],
        'D20':[0.708, 0.761],
        'D40':[0.632, 0.672 ]}

f1_score(dict)

D00 0.672
D10 0.584
D20 0.734
D40 0.651
0.66


## Grid Search


In [ ]:
grid_param = {
    'weights' : ['IMSC', 'small', 'nano'],
    'freeze' : [0, 12, 18],
   'batch_size': [16,32,64],
    'epochs': [40, 80, 120]
}

In [ ]:
import itertools
grid = []
for i in itertools.product(grid_param['weights'],grid_param['freeze'], grid_param['batch_size'], grid_param['epochs']):
  grid.append(i)

In [ ]:
for comb in grid:
  if comb[0] == 'IMSC':
    model = "weights/IMSC/last_95.pt"
  elif comb[0] == 'small':
    model = 'yolov5s.pt'
  elif comb[0] == 'nano':
    model = 'yolov5n.pt'
  else:
    break

  fr = comb[1]
  bs = comb[2]
  ep = comb[3]

  name = '-'+comb[0]+'-'+str(comb[1])+'-'+str(comb[2])+'-'+str(comb[3])
  best_model = f"runs/train/exp{name}/weights/best.pt"

  !python3 train.py --data data/road_col_extended.yml --weights $model --img 640 --batch-size $bs --epochs $ep --freeze $fr --name exp$name

  !python val.py --weights $best_model  --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col_extended.yml --augment --half --task test --name expTest$name

  !python val.py --weights $best_model --img 640 --data /content/gdrive/MyDrive/ultralytics/yolov5/data/road_col_extended.yml --augment --half --task train --name expTrain$name

## Experimento 17

In [ ]:
!python3 train.py --data data/road_col_extended2.yml --weights weights/IMSC/last_95.pt --img 640 --batch-size 32 --epochs 80 --freeze 12

train: weights=weights/IMSC/last_95.pt, cfg=, data=data/road_col_extended2.yml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=80, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[12], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
Command 'git fetch && git config --get remote.origin.url' timed out after 5 seconds
fatal: cannot change to '/content/gdrive/My': No such file or directory
YOLOv5 🚀 2022-6-28 Python-3.7.15 torch-1.12.1+cu113 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_